# Configuration

In [9]:
import os
import time
import json
import numpy as np
from collections import defaultdict
from urllib import request, parse
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

__file__ = os.getcwd()

In [ ]:
FDIR_DATA = os.path.join(__file__, 'data')

FDIR_URL_LIST = os.path.join(FDIR_DATA, 'news/hyundai_motors/url_list/')
make_dir_soft(FDIR_URL_LIST)

FDIR_ARTICLE = os.path.join(FDIR_DATA, 'news/hyundai_motors/article/')
make_dir_soft(FDIR_ARTICLE)

# Web Crawling

In [13]:
def make_dir_soft(fpath):
    dirpath = os.path.dirname(fpath)
    if os.path.isdir(dirpath):
        pass
    else:
        os.makedirs(dirpath)
        
    return None

def remove_date_sep(date):
    return data.replace('.', '')

def calculate_date_list(date_start, duration):
    date_start_by_datetime = datetime.strptime(date_start, '%Y.%m.%d')

    date_list = []
    for days_after in range(duration):
        date_end_by_datetime = date_start_by_datetime + timedelta(days=days_after)
        date_end = datetime.strftime(date_end_by_datetime, '%Y.%m.%d')

        date_list.append(date_end)
        
    return date_list

def request_for_soup(url, headers):
    req = request.Request(url=url, headers=headers)
    html = request.urlopen(req).read()
    soup = BeautifulSoup(html, 'lxml')
    
    return soup

def is_list_page_empty(soup):
    if soup.find('div', class_='not_found02'):
        return True
    else:
        return False
    
def get_sleep_time_random():
    random_number = np.random.normal(1,0.1)
    if random_number > 0:
        pass
    elif random_number < 0:
        random_number = random_number * (-1)
    else:
        random_number = 1
    
    return random_number

def parse_article(soup):
    title = soup.find('h2').get_text()
    date, _, _ = soup.find('span', class_='media_end_head_info_datestamp_time').get_text().split()
    content = soup.find('div', id='dic_area').get_text()
    
    return title, date, content

def import_url_dict(fdir):
    url_dict = {}
    for fname in os.listdir(fdir):
        date, _ = fname.split('.')
        
        fpath = os.path.join(fdir, fname)
        with open(fpath, 'r', encoding='utf-8') as f:
            url_dict[date] = json.load(f)
            
    return url_dict

In [4]:
url_base = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={}&sort=1&photo=0&field=0&pd=3&ds={}&de={}&start={}'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"}

QUERY = '현대자동차'
QUERY_PARSED = parse.quote(QUERY)

DATE_START = '2023.07.01'
DURATION = 30
DATE_LIST = calculate_date_list(DATE_START, DURATION)

print('Query parsed: {}'.format(QUERY_PARSED))
print('Date list: {}'.format('\n'.join(DATE_LIST)))

Query parsed: %ED%98%84%EB%8C%80%EC%9E%90%EB%8F%99%EC%B0%A8
Date list: 2023.07.01
2023.07.02
2023.07.03
2023.07.04
2023.07.05
2023.07.06
2023.07.07
2023.07.08
2023.07.09
2023.07.10
2023.07.11
2023.07.12
2023.07.13
2023.07.14
2023.07.15
2023.07.16
2023.07.17
2023.07.18
2023.07.19
2023.07.20
2023.07.21
2023.07.22
2023.07.23
2023.07.24
2023.07.25
2023.07.26
2023.07.27
2023.07.28
2023.07.29
2023.07.30


In [109]:
URL_COUNT = 0

for date in DATE_LIST:
    print('--'*30)
    print('Date: {}'.format(date))
    
    # Init URL list and start index for a new day
    url_list = []
    URL_START_IDX = 1
    
    while True:
        # Get URL
        url_list_page = url_base.format(QUERY_PARSED, date, date, URL_START_IDX)

        # Parse HTML
        soup = request_for_soup(url_list_page, headers)
        if is_list_page_empty(soup):
            break
        else:
            pass
        
        # Extend URL list of the current date
        url_list.extend([url.get('href') for url in soup.find_all('a', class_='info') if '네이버뉴스' in url])
        
        # Sleep and move to next page of the URLs
        SLEEP_TIME = get_sleep_time_random()
        time.sleep(SLEEP_TIME)
        URL_START_IDX += 10
    
    # Export URL list
    date_for_dirname = remove_date_sep(date)
    FNAME_URL_LIST = '{}.json'.format(date_for_dirname)
    FPATH_URL_LIST = os.path.join(FDIR_URL_LIST, FNAME_URL_LIST)
    with open(FPATH_URL_LIST, 'w', encoding='utf-8') as f:
        json.dump(url_list, f)
        
    # Extend total URL list
    URL_COUNT += len(url_list)
    print('URL list (current day): {}'.format(len(url_list)))
    print('URL list (total)      : {}'.format(URL_COUNT))
    
print('=='*30)
print('Done: {}'.format(URL_COUNT))

------------------------------------------------------------
Date: 2023.07.01
URL list (current day): 21
URL list (total)      : 21
------------------------------------------------------------
Date: 2023.07.02
URL list (current day): 62
URL list (total)      : 83
------------------------------------------------------------
Date: 2023.07.03
URL list (current day): 193
URL list (total)      : 276
------------------------------------------------------------
Date: 2023.07.04
URL list (current day): 153
URL list (total)      : 429
------------------------------------------------------------
Date: 2023.07.05
URL list (current day): 223
URL list (total)      : 652
------------------------------------------------------------
Date: 2023.07.06
URL list (current day): 126
URL list (total)      : 778
------------------------------------------------------------
Date: 2023.07.07
URL list (current day): 47
URL list (total)      : 825
------------------------------------------------------------
Date: 

In [12]:
url_dict = import_url_dict(FDIR_URL_LIST)

url_count = len(url_dict.values())
article_count_total = 0
errors = []

for url_date, url_list in url_dict.items():
    article_count_date = 0
    for url_article in url_list:
        # Check existence
        article_id = '{:04d}'.format(article_count_date)
        FNAME_ARTICLE = '{}/{}.json'.format(remove_date_sep(url_date), article_id)
        FPATH_ARTICLE = os.path.join(FDIR_ARTICLE, FNAME_ARTICLE)
        
        if os.path.isfile(FPATH_ARTICLE):
            article_count_total += 1
            continue
        else:
            pass
        
        # Parse HTML
        soup = request_for_soup(url_article, headers)

        # Parse article
        try:
            title, date, content = parse_article(soup)
        except:
            errors.append(url_article)
            continue

        article = {
            'title': title,
            'date': date,
            'content': content
        }

        # Save article
        make_dir_soft(FPATH_ARTICLE)
        with open(FPATH_ARTICLE, 'w', encoding='utf-8') as f:
            json.dump(article, f)
            article_count_date += 1
            article_count_total += 1
            
        # Sleep and move to next page of the URLs
        SLEEP_TIME = get_sleep_time_random()
        time.sleep(SLEEP_TIME)

    # Print status
    print('Date: {} / Total Success: {:,} / Errors: {:,}'.format(url_date, article_count_total, len(errors)))
    
# Print result
print('=='*30)
print('Done')
print('Total   : {:,}'.format(url_count))
print('Articles: {:,}'.format(article_count_total))
print('Errors  : {:,}'.format(len(errors)))
print('--'*30)
print(errors)

Date: 20230701 / Total Success: 21 / Errors: 0
Date: 20230702 / Total Success: 83 / Errors: 0
Date: 20230703 / Total Success: 276 / Errors: 0
Date: 20230704 / Total Success: 429 / Errors: 0
Date: 20230705 / Total Success: 652 / Errors: 0
Date: 20230706 / Total Success: 774 / Errors: 4
Date: 20230707 / Total Success: 818 / Errors: 7
Date: 20230708 / Total Success: 835 / Errors: 13
Date: 20230709 / Total Success: 905 / Errors: 14
Date: 20230710 / Total Success: 1,051 / Errors: 17
Date: 20230711 / Total Success: 1,164 / Errors: 22
Date: 20230712 / Total Success: 1,292 / Errors: 23
Date: 20230713 / Total Success: 1,508 / Errors: 25
Date: 20230714 / Total Success: 1,604 / Errors: 25
Date: 20230715 / Total Success: 1,612 / Errors: 25
Date: 20230716 / Total Success: 1,662 / Errors: 25
Date: 20230717 / Total Success: 1,716 / Errors: 25
Date: 20230718 / Total Success: 1,852 / Errors: 25
Date: 20230719 / Total Success: 1,949 / Errors: 27
Date: 20230720 / Total Success: 2,161 / Errors: 27
Date: 2

# Data Partition

In [15]:
from sklearn.model_selection import train_test_split

In [103]:
def iter_fpath_article(fdir_article):
    for date in os.listdir(fdir_article):
        fdir_article_date = os.path.join(fdir_article, date)
        flist_article_date = os.listdir(fdir_article_date)
        for idx, fname in enumerate(flist_article_date):
            fpath = os.path.join(fdir_article_date, fname)
            yield fpath
            
def load_articles(fdir_article):
    articles = []
    for fpath in iter_fpath_article(fdir_article):
        article_id, _ = '/'.join(fpath.replace('\\', '/').split('/')[-2:]).split('.')
        with open(fpath, 'r', encoding='utf-8') as f:
            article = json.load(f)
            article['id'] = article_id
            
            articles.append(article)
            
    return articles

In [104]:
articles = load_articles(FDIR_ARTICLE)
print(len(articles))
print(articles[:3])

2976
[{'title': '[단독] 현대차 직원은 왜 아파트 주차장에 숨었을까', 'date': '2023.07.01.', 'content': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n [앵커]현대자동차가 외근 영업 사원의 근태를 확인하겠다며 집앞까지 사람을 보내 \'몰래 촬영\'을 해온 사실이 확인됐습니다.차 안에 숨어서 직원이 몇 시에 집에 들어가는지, 언제 나오는지, 매일 찍도록 한 건데, 일종의 \'사생활\'을 감시한 이 촬영에 대해서, 법원은 \'그럴 만한 이유가 있었다, 위법하지 않다\'는 취지의 판결을 내렸습니다.여러분은 어떻게 보시는지요?KBS가 여러 달 추적 끝에 입수한 촬영물을 공개해 드리니, 이 영상 보시고, 같이 한 번 생각해 보시면 좋겠습니다.백인성 법조전문 기자입니다. [리포트] 아파트 단지를 걸어가는 한 여성. 현관에 들어서기까지 누군가가 자동차 안에서  영상을 찍습니다. 2020년 3월 9일 3시 28분, 촬영 시작 시간도 기록합니다. 외근이 잦은 판매 직원 A 씨의 근태를 확인하겠다며  현대자동차 직원이 몰래 찍은 영상입니다. 주말을 빼고는 매일 아파트 앞을 지킨 탓에 주차 단속 딱지까지 붙었지만,  촬영은 2개월 간 계속됐습니다. 이후 현대차는 A 씨가 근무시간 중  매일 세 시간 넘게 집에 머물렀다며 해고했습니다. A 씨는 소송 과정에서야 회사의 몰래 촬영 사실을 알게 됐습니다.   [A 씨 남편 : "전혀 몰랐죠. 아무도 몰랐어요. (주민) 수백 명이 찍혔는데 아무도, 그 바로 앞에 찍혔는데도 아무도 몰랐어요. 실질적으로 이렇게 심각하게 저희를 사찰하고 있을 거라고 생각을 못했고…"] 현대차 측의 잠복 촬영은 A 씨의 근태 제보에 대한  \'현장 조사\' 명목이었고,  A 씨가 낸 해고무효 소송에  증거 자료로 제출됐습니다. [노동 전문 변호사/음성변조 : "주변 사람 사실확인서라든지 진술서같은 걸 통해서 (근무 태만을) 입증을 하는데, 이렇게까지 하는 경우는 처음 본다."] A 씨 측은 재판에서 현대차가 

In [69]:
np.random.seed(42)
RANDOM_INT_MIN = 0
RANDOM_INT_MAX = 3
LABEL_LEN = len(articles)

labels = np.random.randint(0, 3, size=LABEL_LEN)
print(len(labels))
print(labels[:30])

2976
[2 0 2 2 0 0 2 1 2 2 2 2 0 2 1 0 1 1 1 1 0 0 1 1 0 0 0 2 2 2]


train_test_split

- test_size: 전체 데이터에서 test 데이터의 비율
- random_state: 데이터를 train과 test로 나누는 방식 --> 동일한 random_state라면 동일하게 구분됨

In [74]:
X_train, X_test, y_train, y_test = train_test_split(articles, labels, test_size=0.2, random_state=42)

print('# of X_train: {:,}'.format(len(X_train)))
print('# of X_test: {:,}'.format(len(X_test)))
print('# of y_train: {:,}'.format(len(y_train)))
print('# of y_test: {:,}'.format(len(y_test)))

# of X_train: 2,380
# of X_test: 596
# of y_train: 2,380
# of y_test: 596


# Data Labeling

데이터 라벨링을 해봅시다.
- 1: 자동차 판매 관련 기사
- 0: 그 외

In [75]:
import pandas as pd

In [122]:
date_list = sorted(set([article['date'].replace('.', '') for article in articles]))
date_start, date_end = date_list[0], date_list[-1]

print('Date list: {}'.format(date_list))
print('--'*30)
print('Date start: {} / Date end: {}'.format(date_start, date_end))

Date list: ['20230701', '20230702', '20230703', '20230704', '20230705', '20230706', '20230707', '20230708', '20230709', '20230710', '20230711', '20230712', '20230713', '20230714', '20230715', '20230716', '20230717', '20230718', '20230719', '20230720', '20230721', '20230722', '20230723', '20230724', '20230725', '20230726', '20230727', '20230728', '20230729', '20230730']
------------------------------------------------------------
Date start: 20230701 / Date end: 20230730


In [125]:
FDIR_DATA_LABELING = os.path.join(__file__, 'corpus/news/hyundai_motors')
FNAME_DATA_LABELING = 'from{}to{}_before.xlsx'.format(date_start, date_end)

In [106]:
data = {'id': [],
        'title': [],
        'date': [],
        'content': [],
        'label': []
       }

for article in articles:
    title, date, content, _id = article.values()
    label_init = 0
    
    data['id'].append(_id)
    data['title'].append(title)
    data['date'].append(date)
    data['content'].append(content)
    data['label'].append(label_init)
    
data

{'id': ['20230701/0000',
  '20230702/0000',
  '20230702/0001',
  '20230702/0002',
  '20230702/0003',
  '20230702/0004',
  '20230702/0005',
  '20230702/0006',
  '20230702/0007',
  '20230702/0008',
  '20230702/0009',
  '20230702/0010',
  '20230702/0011',
  '20230702/0012',
  '20230702/0013',
  '20230702/0014',
  '20230702/0015',
  '20230702/0016',
  '20230702/0017',
  '20230702/0018',
  '20230702/0019',
  '20230702/0020',
  '20230702/0021',
  '20230702/0022',
  '20230702/0023',
  '20230702/0024',
  '20230702/0025',
  '20230702/0026',
  '20230702/0027',
  '20230702/0028',
  '20230702/0029',
  '20230702/0030',
  '20230702/0031',
  '20230702/0032',
  '20230702/0033',
  '20230702/0034',
  '20230702/0035',
  '20230702/0036',
  '20230702/0037',
  '20230702/0038',
  '20230702/0039',
  '20230702/0040',
  '20230702/0041',
  '20230702/0042',
  '20230702/0043',
  '20230702/0044',
  '20230702/0045',
  '20230702/0046',
  '20230702/0047',
  '20230702/0048',
  '20230702/0049',
  '20230702/0050',
  '202

In [108]:
data_df = pd.DataFrame(data)
data_df

,id,title,date,content,label
0,20230701/0000,[단독] 현대차 직원은 왜 아파트 주차장에 숨었을까,2023.07.01.,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n [앵커]현대자동차가 외근 영...,0
1,20230702/0000,"[사설] “정권 퇴진, 오염수 반대” 민노총 정치파업 엄정 대응해야",2023.07.02.,\n \n\n\n\n 전국민주노동조합총연맹(민노총)이 오늘부터 15일까지 2주간 ...,0
2,20230702/0001,미국서 사고 싶은 브랜드 됐다…79만대 팔린 한국차 ‘최대 실적’,2023.07.02.,\n현대차·기아 상반기 신차판매종전 최고실적 2021년 넘어양사 SUV 판매비중 7...,0
3,20230702/0002,"아직은, 엔진 가치를 증명하다…아우디 고성능 내연차 RS7 스포트백·RS6 아반트 시승기",2023.07.02.,\n\n\n\n\n아우디의 고성능 세단인 ‘RS7 스포트백’은 최대출력 630마력에...,0
4,20230702/0003,‘궁극의 친환경’ 수소차…상용화까지 과제는?,2023.07.02.,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n [앵커] 친환경차 하면 주로...,0
...,...,...,...,...,...
2971,20230730/0004,"현대차그룹, 상반기 美 시장 점유율 10.6%…스텔란티스 제치고 4위",2023.07.30.,\n\t\t\t현대자동차그룹이 올해 상반기 미국 자동차 시장에서 점유율을 끌어올리며...,0
2972,20230730/0005,"전기차 충전, 알아두면 좋은 몇가지 상식들…",2023.07.30.,\n[테크따라잡기] 내 차에는 어떤 배터리가…'유형별 특징은'급속충전기 제각각 이라...,0
2973,20230730/0006,"“자동차 사볼까” 구매 욕구 1년 만에 최고치, 왜? [여車저車]",2023.07.30.,\n유가하락·저가 전기차 등장으로 가격 부담↓공급망 문제 해소로 출고대기 기간도 짧...,0
2974,20230730/0007,"[직장생활백서]""반바지 입으면 안 되나요"" '복장 자율화' 당신의 생각은",2023.07.30.,"\n기업들, 폭염 속 쿨비즈 패션 권장""눈치 보여 못입겠다"" 회사 방침에도 하소연최...",0


In [127]:
fpath_data_labeling = os.path.join(FDIR_DATA_LABELING, FNAME_DATA_LABELING)
make_dir_soft(fpath_data_labeling)

data_df.to_excel(fpath_data_labeling)
print('Export data: {}'.format(fpath_data_labeling))

Export data: C:\Google Drive\playground\23_workspace\bug\corpus/news/hyundai_motors\from20230701to20230730_before.xlsx


자동차 판매 관련 기사의 label을 1로 변경한 뒤 "\~\~\_after_<이름>.xlsx"의 파일명으로 저장
- 김가영: 0~99
- 김서연: 100~199
- 김현태: 200~299
- 민경제: 300~399
- 박경훈: 400~499
- 박규리: 500~599
- 박유민: 600~699
- 변성준: 700~799
- 서유탁: 800~899
- 서준혁: 900~999
- 성무선: 1,000~1,099
- 성해준: 1,100~1,199
- 신서빈: 1,200~1,299
- 이석희: 1,300~1,399
- 이주연: 1,400~1,499
- 한승우: 1,500~1,599
- 홍정화: 1,600~1,699

---
파일명 예시: "\~/from20230701to20230730_after_홍길동.xlsx"